In [1]:
import pandas as pd
import secret
import json
import os
import pickle
import re

from graphrag_for_all.llm.openai import set_openai_api_key
from graphrag_for_all.llm.huggingface import set_hugging_face_token
from graphrag_for_all.llm.create import get_send_fn
from utils.query import get_questions_by_lesion
from graphrag_for_all.search.searcher import Searcher

set_openai_api_key(secret.OPENAI_API_KEY)
set_hugging_face_token(secret.HUGGINGFACE_TOKEN)
send_fn = get_send_fn(source="huggingface", model_name="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
with open("./combined_index_results/graphrag/index_graphrag_llama3v1_combined_top_1/separated_extracted_keywords.pkl", "rb") as f:
    separated_extracted_keywords = pickle.load(f)

In [3]:
separated_extracted_keywords

{'responses': OrderedDict([('enlarged cardiac silhouette',
               OrderedDict([('What are the symptoms associated with enlarged cardiac silhouette?',
                             "**Symptoms Associated with Enlarged Cardiac Silhouette**\n===========================================================\n\nAn enlarged cardiac silhouette is a condition where the heart appears larger than normal on an X-ray or other imaging tests. This can be caused by various factors, including heart failure, cardiomyopathy, or other conditions that affect the heart's structure or function.\n\n**Common Symptoms**\n-------------------\n\nAccording to the available data, the following symptoms are commonly associated with an enlarged cardiac silhouette:\n\n*   **Shortness of breath**: This is one of the most common symptoms, indicating that the heart is not pumping efficiently and is causing fluid buildup in the lungs [Data: Reports (2, 7, 64, 46, 34, +more)].\n*   **Fatigue**: Feeling tired or weak is a

In [4]:
extracted_keywords_results = separated_extracted_keywords

In [5]:
MIMIC_EYE_PATH = "F:\\mimic-eye"
REFLACX_LESION_LABEL_COLS = [
    # "Fibrosis",
    # "Quality issue",
    # "Wide mediastinum",
    # "Fracture",
    # "Airway wall thickening",
    ######################
    # "Hiatal hernia",
    # "Acute fracture",
    # "Interstitial lung disease",
    # "Enlarged hilum",
    # "Abnormal mediastinal contour",
    # "High lung volume / emphysema",
    # "Pneumothorax",
    # "Lung nodule or mass",
    # "Groundglass opacity",
    ######################
    "Pulmonary edema",
    "Enlarged cardiac silhouette",
    "Consolidation",
    "Atelectasis",
    "Pleural abnormality",
    # "Support devices",
]

In [2]:
import pickle

In [3]:
with open("./combined_index_results/graphrag/index_graphrag_llama3v1_combined_top_1/combined_extracted_keywords.pkl", "rb") as f:
    combined_extracted_keywords = pickle.load(f)

In [4]:
combined_extracted_keywords

{'responses': OrderedDict([('enlarged cardiac silhouette',
               OrderedDict([('What are the symptoms associated with enlarged cardiac silhouette?',
                             "**Symptoms Associated with Enlarged Cardiac Silhouette**\n===========================================================\n\nAn enlarged cardiac silhouette is a condition where the heart appears larger than normal on an X-ray or other imaging tests. This can be caused by various factors, including heart failure, cardiomyopathy, or other conditions that affect the heart's structure or function.\n\n**Common Symptoms**\n-------------------\n\nAccording to the available data, the following symptoms are commonly associated with an enlarged cardiac silhouette:\n\n*   **Shortness of breath**: This is one of the most common symptoms, indicating that the heart is not pumping efficiently and is causing fluid buildup in the lungs [Data: Reports (2, 7, 64, 46, 34, +more)].\n*   **Fatigue**: Feeling tired or weak is a

In [ ]:
# "./combined_index_results/graphrag/index_graphrag_llama3v1_combined_top_1/combined_extracted_keywords.pkl"

# # knowledge_graph_dir = os.path.join(
# #     /combined_extracted_keywords.pkl

# #     f"./{args.output_dir}", args.store_type, f"/{lesion}_top_{args.doc_top_k}"

# # )

In [6]:
# "./combined_index_results/graphrag/index_graphrag_llama3v1_combined_top_1/combined_extracted_keywords.pkl"

# # knowledge_graph_dir = os.path.join(
# #     /combined_extracted_keywords.pkl

# #     f"./{args.output_dir}", args.store_type, f"/{lesion}_top_{args.doc_top_k}"

# # )


searcher = Searcher(
    input_dir="./combined_index_results/graphrag/index_graphrag_llama3v1_combined_top_1/",
    send_to=send_fn,
    community_level=1,

)

f:\GitHub\rag-aug\graphrag_for_all\search\read.py:1099: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
f:\GitHub\rag-aug\graphrag_for_all\search\read.py:1100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


In [7]:
from collections import OrderedDict
def build_prior_knowledge(extracted_keywords_results):
    lesion_qa_pairs = extracted_keywords_results['responses']

    prior_knowledge = OrderedDict({})
    for lesion, q_a in lesion_qa_pairs.items():
        q_a_section = ""
        for q, a in q_a.items():
            q_a_section += f"\n#############################################\n**Question**: {q}\n**Answer**:\n{a}\n"
        lesion_content = f"## Lesion: {lesion}\n" + q_a_section
        prior_knowledge[lesion] = lesion_content
    return prior_knowledge

In [8]:
prior_knowledge = build_prior_knowledge(extracted_keywords_results)
all_prior_knowledge = "\n\n\n".join(prior_knowledge.values())

In [9]:
requesting_prompt = f""" The following is the information from {len(prior_knowledge)} lesions, including {", ".join(list(prior_knowledge.keys()))}. Please combine and summarize them.

{all_prior_knowledge}

(Please return the summarized version directly, without additional text.)

"""

In [10]:
DEFAULT_LLM_ARGS = {
    "temperature": 0.0,
    "top_p": 1.0,
}  #

In [11]:
pk_res = send_fn(
    [
        {"role": "system", "content": "You are a helpful clinical assistant."},
        {"role": "user", "content": requesting_prompt},
    ],
    DEFAULT_LLM_ARGS,
)
# 5m 11.2s

c:\Users\mike8\anaconda3\envs\pytorch\Lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\mike8\anaconda3\envs\pytorch\Lib\site-packages\transformers\models\llama\modeling_llama.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [12]:
print(pk_res.output)

**Summary of Lesions: Enlarged Cardiac Silhouette, Pulmonary Consolidation, Atelectasis, and Pleural Abnormality**

**Enlarged Cardiac Silhouette:**

*   Symptoms: shortness of breath, fatigue, swelling in the legs and feet, chest pain, palpitations, and coughing up blood
*   Causes: pericardial effusion, cardiomegaly, increased cardiothoracic ratio, anterior mediastinal mass, and prominent epicardial fat pad
*   Clinical signs: pericardial effusion, cardiomegaly, increased cardiothoracic ratio, anterior mediastinal mass, and prominent epicardial fat pad
*   Laboratory data: pericardial fluid analysis, radiographic images, echocardiographic images, and biopsy results
*   Personal relevant history: history of pericardial effusion and cardiomegaly, smoking history, and family history of heart disease

**Pulmonary Consolidation:**

*   Symptoms: breath sounds and crackles, chest pain, fever, weakness, shortness of breath, and coughing up mucus or blood
*   Causes: blood from the bronchial

In [13]:
import json

In [14]:
result_json = json.dumps(
    {
        "Respiratory Rate (breaths per minute)": {
            "type": "numerical",
            "healthy range": "12~25",
        },
        "Heart rate (beats per minute)": {
            "type": "numerical",
            "healthy range": "60~100",
        },
        "Infection": {
            "type": "boolean",
        },
    }
)

# IMPORTANT: Ensure that features that indicates the presence or absence of a specific diagnostic finding are boolean data types (ex: **something** showing **something**).
# IMPORTANT: If the feature name contains "showing", changing its data type to boolean.
# If a feature indicates the presence or absence of a specific diagnostic finding, change its data type to boolean. Otherwise, keep the original data type.
# (Please only return the json object without additional text)


In [15]:
lesion_keywords= "\n\n".join([f"## Lesion: {k}\n**Features:**{json.dumps(v)}\n" for k, v in extracted_keywords_results['keywords'].items()])

keyword_combining_prompt = f"""The following json objects are features from {len(extracted_keywords_results['keywords'])} different lesions. The key represents the feature, while the value indicates the data type.

Please refine and combine the following features from {len(extracted_keywords_results['keywords'])} lesions, including {", ".join(list(extracted_keywords_results['keywords'].keys()))}. 

These features will be used to predict diseases and lesions. However, some may have incorrect data types, so please correct them.

Additionally, repetitive or similar features from different lesions should be combined or removed. For numerical values, ensure the unit is placed at the end of each feature, if applicable.

When a feature is a diagnostic finding, change its data type to boolean.

For example: 

```json
{{
  "Lung Volume": "numerical",
  "Heart rate": "numerical",  
  "A showing B": "numerical",
}}
```
Should be:
```
{{
  "Lung Volume (L)": "numerical",
  "Heart rate" (beats per minute): "numerical",  
  "A showing B": "boolean",
}}
```

If it's something like risk factors, please put the range at the end (e.g., 1-5, 1-10)

(Please only return the json object without additional text)

# Features

{lesion_keywords}
"""

refined_keywords_res = send_fn(
    [
        {
            "role": "system",
            "content": f"You are a helpful clinical assistant and has following information in mind:\n{pk_res.output}",
        },
        {"role": "user", "content": keyword_combining_prompt},
    ],
    DEFAULT_LLM_ARGS,
)

In [16]:
print(refined_keywords_res.output)

```json
{
  "Shortness of breath": "boolean",
  "Fatigue": "boolean",
  "Swelling in the legs and feet": "boolean",
  "Chest pain": "boolean",
  "Palpitations": "boolean",
  "Coughing up blood": "boolean",
  "Fluid buildup in the lungs": "boolean",
  "Confusion": "boolean",
  "Loss of appetite": "boolean",
  "Pericardial effusion": "boolean",
  "Cardiomegaly": "boolean",
  "Increased cardiothoracic ratio": "boolean",
  "Anterior mediastinal mass": "boolean",
  "Prominent epicardial fat pad": "boolean",
  "Elevated pericardial fluid levels (mL)": "numerical",
  "Increased pericardial fluid protein levels (mg/dL)": "numerical",
  "Presence of pericardial fluid cells": "boolean",
  "Radiographic images showing an increased cardiothoracic ratio": "boolean",
  "Measurements of cardiac diameter and thoracic diameter (cm)": "numerical",
  "Echocardiographic images showing an enlarged heart": "boolean",
  "Measurements of cardiac dimensions (cm)": "numerical",
  "Imaging studies showing a mass

In [17]:
dataset_features = [
    "Gender",
    "Age",
    "Blood Pressure",
    "Body Temperature",
    "Heart rate",
    "Respiratory Rate",
    "Oxygen Saturation",
    "Age",
    "Gender",
]

dataset_features_str = ", ".join(dataset_features)

res_existing_features = send_fn(
    refined_keywords_res.history
    + [
        {
            "role": "user",
            "content": f"From above refined features, please indicate me the features that are exactly included in: {dataset_features_str}. (Only return a list of related features without additional text)",
        }
    ],
    DEFAULT_LLM_ARGS,
)

In [18]:
print(res_existing_features.output)

[
  "Age (years)",
  "Body Temperature (°C)",
  "Age (years)"
]


In [19]:
# with open("separated_combined_results", "rb") as f:
#     loaded_results = pickle.load(f)

In [20]:
def extract_json_string(text):
    """
    This function extract the last json object in the text, because the responses return repetitive json objects sometimes.
    """
    json_start = text.find("{")
    json_end = text.find("}") + 1

    if json_end > json_start:
        return text[json_start:json_end]
    else:
        raise ValueError(
            f"The end of bracket is earlier than the start of bracket in the response:\n{text}."
        )

In [21]:
refined_keywords_res.output

'```json\n{\n  "Shortness of breath": "boolean",\n  "Fatigue": "boolean",\n  "Swelling in the legs and feet": "boolean",\n  "Chest pain": "boolean",\n  "Palpitations": "boolean",\n  "Coughing up blood": "boolean",\n  "Fluid buildup in the lungs": "boolean",\n  "Confusion": "boolean",\n  "Loss of appetite": "boolean",\n  "Pericardial effusion": "boolean",\n  "Cardiomegaly": "boolean",\n  "Increased cardiothoracic ratio": "boolean",\n  "Anterior mediastinal mass": "boolean",\n  "Prominent epicardial fat pad": "boolean",\n  "Elevated pericardial fluid levels (mL)": "numerical",\n  "Increased pericardial fluid protein levels (mg/dL)": "numerical",\n  "Presence of pericardial fluid cells": "boolean",\n  "Radiographic images showing an increased cardiothoracic ratio": "boolean",\n  "Measurements of cardiac diameter and thoracic diameter (cm)": "numerical",\n  "Echocardiographic images showing an enlarged heart": "boolean",\n  "Measurements of cardiac dimensions (cm)": "numerical",\n  "Imagin

In [22]:
keywords = json.loads(extract_json_string(refined_keywords_res.output))

In [23]:
numerical_keywords = [k for k, v in keywords.items() if v == 'numerical']

In [24]:
numerical_keywords

['Elevated pericardial fluid levels (mL)',
 'Increased pericardial fluid protein levels (mg/dL)',
 'Measurements of cardiac diameter and thoracic diameter (cm)',
 'Measurements of cardiac dimensions (cm)',
 'White Blood Cell Count (cells/μL)',
 'C-Reactive Protein (CRP) levels (mg/L)',
 'Procalcitonin levels (ng/mL)',
 'Lung Volume (L)',
 'Diffusing Capacity (mL/min/mmHg)',
 'Gas Exchange (mL/min/mmHg)',
 'Age (years)',
 'Oxygen levels (mmHg)',
 'Body Temperature (°C)',
 'Breathing Rate (breaths/min)',
 'Cardiothoracic ratio',
 'Lung expansion',
 'Pleural effusion risk factors (1-5)',
 'Pleural disease risk factors (1-10)']

In [25]:
keyword_combining_prompt = f"""

Please specify range of values of features for an health adult.
Please return the json object without additional text.

# Example

## Input
```json
["Respiratory rate (breaths per minute)", "Heart rate (beats per minute)"]
```

## Output
```json
{{
  "Respiratory rate (breaths per minute)": "12-20",
  "Heart rate (beats per minute)": "60-100",  
}}
```

# Real

## Input
```json
{json.dumps(numerical_keywords)}
```

## Output
"""
healthy_numerical_range_res = send_fn(
    [
        {
            "role": "system",
            "content": f"You are a helpful clinical assistant and has following information in mind:\n{pk_res.output}",
        },
        {"role": "user", "content": keyword_combining_prompt},
    ],
    DEFAULT_LLM_ARGS,
)

In [26]:
print(healthy_numerical_range_res.output)

```json
{
  "Elevated pericardial fluid levels (mL)": "0-200",
  "Increased pericardial fluid protein levels (mg/dL)": "0-5",
  "Measurements of cardiac diameter and thoracic diameter (cm)": "15-25",
  "Measurements of cardiac dimensions (cm)": "10-20",
  "White Blood Cell Count (cells/μL)": "4,000-11,000",
  "C-Reactive Protein (CRP) levels (mg/L)": "0-10",
  "Procalcitonin levels (ng/mL)": "0-0.5",
  "Lung Volume (L)": "3-6",
  "Diffusing Capacity (mL/min/mmHg)": "20-40",
  "Gas Exchange (mL/min/mmHg)": "10-20",
  "Age (years)": "18-65",
  "Oxygen levels (mmHg)": "75-100",
  "Body Temperature (°C)": "36-37",
  "Breathing Rate (breaths/min)": "12-20",
  "Cardiothoracic ratio": "0.45-0.55",
  "Lung expansion": "0-10",
  "Pleural effusion risk factors (1-5)": "1-5",
  "Pleural disease risk factors (1-10)": "1-10"
}
```


In [27]:
keyword_combining_prompt = f"""

Please specify range of possible values.
Please return the json object without additional text.

# Example

## Input
```json
["Respiratory rate (breaths per minute)", "Heart rate (beats per minute)"]
```

## Output
```json
{{
  "Respiratory rate (breaths per minute)": "0-28",
  "Heart rate (beats per minute)": "0-300",  
}}
```

# Real

## Input
```json
{json.dumps(numerical_keywords)}
```

## Output
"""
possible_numerical_range_res = send_fn(
    [
        {
            "role": "system",
            "content": f"You are a helpful clinical assistant and has following information in mind:\n{pk_res.output}",
        },
        {"role": "user", "content": keyword_combining_prompt},
    ],
    DEFAULT_LLM_ARGS,
)

In [28]:
print(possible_numerical_range_res.output)

```json
{
  "Elevated pericardial fluid levels (mL)": "0-1000",
  "Increased pericardial fluid protein levels (mg/dL)": "0-5",
  "Measurements of cardiac diameter and thoracic diameter (cm)": "0-30",
  "Measurements of cardiac dimensions (cm)": "0-20",
  "White Blood Cell Count (cells/μL)": "0-100,000",
  "C-Reactive Protein (CRP) levels (mg/L)": "0-100",
  "Procalcitonin levels (ng/mL)": "0-50",
  "Lung Volume (L)": "0-10",
  "Diffusing Capacity (mL/min/mmHg)": "0-1000",
  "Gas Exchange (mL/min/mmHg)": "0-1000",
  "Age (years)": "0-120",
  "Oxygen levels (mmHg)": "0-100",
  "Body Temperature (°C)": "0-45",
  "Breathing Rate (breaths/min)": "0-40",
  "Cardiothoracic ratio": "0-1.5",
  "Lung expansion": "0-100",
  "Pleural effusion risk factors (1-5)": "1-5",
  "Pleural disease risk factors (1-10)": "1-10"
}
```


In [30]:
with open("separated_combined_results", "wb") as f:
    pickle.dump(
        {
            "prior_knowledge": pk_res,
            "refined_keyword": refined_keywords_res,
            "existing_features": res_existing_features,
            "healthy_numerical_range_res": healthy_numerical_range_res,
            "possible_numerical_range_res": possible_numerical_range_res,
        },
        f,
    )

In [31]:
prompt = 'You are a clinical expert. With following extra knowledge in mind:\n\n**Summary of Lesions: Enlarged Cardiac Silhouette, Pulmonary Consolidation, Atelectasis, and Pleural Abnormality**\n\n**Enlarged Cardiac Silhouette:**\n\n*   Symptoms: shortness of breath, fatigue, swelling in the legs and feet, chest pain, palpitations, and coughing up blood\n*   Causes: pericardial effusion, cardiomegaly, increased cardiothoracic ratio, anterior mediastinal mass, and prominent epicardial fat pad\n*   Clinical signs: pericardial effusion, cardiomegaly, increased cardiothoracic ratio, anterior mediastinal mass, and prominent epicardial fat pad\n*   Laboratory data: pericardial fluid analysis, radiographic images, echocardiographic images, and biopsy results\n*   Personal relevant history: history of pericardial effusion and cardiomegaly, smoking history, and family history of heart disease\n\n**Pulmonary Consolidation:**\n\n*   Symptoms: breath sounds and crackles, chest pain, fever, weakness, shortness of breath, and coughing up mucus or blood\n*   Causes: blood from the bronchial tree and hemorrhage from the pulmonary artery, pulmonary edema, pus, and pulmonary infiltrate\n*   Clinical signs: breath sounds and crackles, blood from the bronchial tree and hemorrhage from the pulmonary artery, pulmonary edema, pus, and pulmonary infiltrate\n*   Laboratory data: elevated white blood cell count, elevated C-reactive protein (CRP) levels, and elevated procalcitonin levels\n*   Personal relevant history: smoking history, chronic obstructive pulmonary disease (COPD), and family history of respiratory conditions\n\n**Atelectasis:**\n\n*   Symptoms: shortness of breath, coughing, chest pain, fatigue, and weight loss\n*   Causes: pneumothorax, pleural effusion, lung cancer, and surfactant deficiency\n*   Clinical signs: lung collapse, increased cardiothoracic ratio, pleural effusion, and pneumothorax\n*   Laboratory data: chest radiography, computed tomography (CT) scans, and pulmonary function tests\n*   Personal relevant history: smoking history, age, previous surgical history, and respiratory conditions\n\n**Pleural Abnormality:**\n\n*   Symptoms: chest pain, shortness of breath, coughing, fatigue, and weight loss\n*   Causes: pleural effusion, pneumoperitoneum, atelectasis, tension pneumothorax, and symptomatic pneumothorax\n*   Clinical signs: pleural effusion, atelectasis, tension pneumothorax, and pneumoperitoneum\n*   Laboratory data: pleural fluid analysis, blood tests, chest radiography, CT scans, and pulmonary function tests\n*   Personal relevant history: smoking history, lung cancer or other malignancies, medical history, family history, and occupational and environmental history\n\n# Task you\'re going to perform\n\nYou will be given a report regarding a patient, and a json object with for you to sepculate the values of each attribute. \nYou will return the speculated values according to the data type specified in the give JSON Object. You need to speculate the value, and null value is not acceptable.\nPlease keep your responses concise, with a maximum length of 1000 tokens.\nDon\'t be unit (e.g., cm, L) in the value. Only numerical characters for numerical data types, and only true or false for boolean data types.\n\nFollowing is an example for you:\n\n## Report\nINDICATION:  Central venous line placement.\nTECHNIQUE:  Frontal chest radiograph.\nCOMPARISON:  Chest radiograph 12:42 today.\nFINDINGS: \nA right subclavian catheter has been placed in the interim. The catheterterminates at the confluence of the brachiocephalic vein and superior venacava and if indicated could be advanced 3.7 cm for termination within thelow SVC.\nThere is no pleural effusion or pneumothorax. The cardiac silhouette remainsmildly enlarged. There is no focal airspace consolidation worrisome forpneumonia.\nHigh density material is again seen in the paritally imaged colon in the leftabdomen. Cholecystectomy clips are noted. There are carotid calcificationsleft greater than right.\nDIAGNOSED LESIONS: Enlarged cardiac silhouette.\nAGE: 69.\nGENDER: Female.\n\n## Json Object \n{"Chest pain": "boolean", "Weight loss": "boolean", "History of COPD": "boolean", "Heartrate": "numerical", "Age": "numerical", "Oxygen levels (%)": "numerical"}\n\n## Expected Output\n{"Chest pain": true, "Weight loss": false, "History of COPD": true, "Age": 69, "Heartrate": 90, "Oxygen levels (%)": 99.0}\n\n\nBased on the following report, determine the potential values for the features listed in the \'Data Type\' JSON object, which specifies the required features and their data types. Speculate on the values rather than stating \'unknown\' or \'not mentioned,\' and return them in JSON format, following the data types provided in the \'Data Type\' JSON object. Avoid generating repetitive results.\n\nEnsure the output complies with the specific JSON format outlined in the system prompt, with the returned JSON object included as a JSON string under the \'description\' field within the \'points\' object.\n\nFor numerical features, refer to the \'Healthy Patient Feature Range\' JSON object for guidance. However, the values do not need to fall strictly within these ranges, as not all patients are healthy—just ensure the values are realistic.\n\n## IMPORTANT\n- Please ensure that all keywords in the JSON object are not missed.\n- Return actual values instead of data types.\n- Please keep your responses concise, with a maximum length of 1000 tokens.\n- If uncertain, speculate the value rather than stating that you cannot answer.\n\n## Report\nINDICATION:  Central venous line placement.\nTECHNIQUE:  Frontal chest radiograph.\nCOMPARISON:  Chest radiograph 12:42 today.\nFINDINGS: \nA right subclavian catheter has been placed in the interim. The catheterterminates at the confluence of the brachiocephalic vein and superior venacava and if indicated could be advanced 3.7 cm for termination within thelow SVC.\nThere is no pleural effusion or pneumothorax. The cardiac silhouette remainsmildly enlarged. There is no focal airspace consolidation worrisome forpneumonia.\nHigh density material is again seen in the paritally imaged colon in the leftabdomen. Cholecystectomy clips are noted. There are carotid calcificationsleft greater than right.\nDIAGNOSED LESIONS: Enlarged cardiac silhouette.\nAGE: 69.\nGENDER: F.\nBODY TEMPERATURE FAHRENHEIT: 98.1.\nHEART RATE PER MINUTE: 90.0.\nRESPIRATORY RATE PER MINUTE: 18.0.\nOXYGEN SATURATION PERCENTAGE: 99.0.\nSYSTOLIC BLOOD PRESSURE mmHg: 184.0.\nSYSTOLIC BLOOD PRESSURE mmHg: 75.0.\nPain 010: 75.0.\nAcuity 1highest priority  5lowest priority: 3.0.\n\n\n\n## Data Type \n{"Shortness of breath": "boolean", "Fatigue": "boolean", "Swelling in the legs and feet": "boolean", "Chest pain": "boolean", "Palpitations": "boolean", "Coughing up blood": "boolean", "Fluid buildup in the lungs": "boolean", "Confusion": "boolean", "Loss of appetite": "boolean", "Pericardial effusion": "boolean", "Cardiomegaly": "boolean", "Increased cardiothoracic ratio": "boolean", "Anterior mediastinal mass": "boolean", "Prominent epicardial fat pad": "boolean", "Elevated pericardial fluid levels (mL)": "numerical", "Increased pericardial fluid protein levels (mg/dL)": "numerical", "Presence of pericardial fluid cells": "boolean", "Radiographic images showing an increased cardiothoracic ratio": "boolean", "Measurements of cardiac diameter and thoracic diameter (cm)": "numerical", "Echocardiographic images showing an enlarged heart": "boolean", "Measurements of cardiac dimensions (cm)": "numerical", "Imaging studies showing a mass in the anterior mediastinum": "boolean", "Biopsy results showing the type of tumor or mass": "boolean", "History of pericardial effusion and cardiomegaly": "boolean", "History of anterior mediastinal mass and prominent epicardial fat pad": "boolean", "History of fluid accumulation": "boolean", "History of air accumulation": "boolean", "History of interstitial lung disease": "boolean", "Smoking History": "boolean", "COPD": "boolean", "Chronic Obstructive Pulmonary Disease": "boolean", "Medical History": "boolean", "Cardiovascular Conditions": "boolean", "Family History": "boolean", "Blood from Bronchial Tree": "boolean", "Hemorrhage from Pulmonary Artery": "boolean", "Pulmonary Edema": "boolean", "Pus": "boolean", "Pulmonary Infiltrate": "boolean", "Infection": "boolean", "Inflammation": "boolean", "Underlying Medical Conditions": "boolean", "White Blood Cell Count (cells/\\u03bcL)": "numerical", "C-Reactive Protein (CRP) levels (mg/L)": "numerical", "Procalcitonin levels (ng/mL)": "numerical", "Lung Volume (L)": "numerical", "Diffusing Capacity (mL/min/mmHg)": "numerical", "Gas Exchange (mL/min/mmHg)": "numerical", "Arterial Blood Gas (ABG) analysis": "boolean", "Complete Blood Count (CBC)": "boolean", "Electrolyte Panel": "boolean", "Breath Sounds and Crackles": "boolean", "Cough": "boolean", "Fever": "boolean", "Difficulty breathing": "boolean", "Wheezing": "boolean", "Coughing up mucus": "boolean", "Difficulty swallowing": "boolean", "Age (years)": "numerical", "Oxygen levels (mmHg)": "numerical", "Body Temperature (\\u00b0C)": "numerical", "Respiratory Symptoms": "boolean", "Coughing": "boolean", "Pleuritic chest pain": "boolean", "Respiratory failure": "boolean", "Breathing Rate (breaths/min)": "numerical", "Previous Surgical History": "boolean", "Respiratory Conditions": "boolean", "Infant Respiratory Distress Syndrome": "boolean", "Pneumothorax": "boolean", "Pleural Effusion": "boolean", "Lung Cancer": "boolean", "Pulmonary Tuberculosis": "boolean", "Postsurgical Atelectasis": "boolean", "Surfactant Deficiency": "boolean", "Cardiothoracic ratio": "numerical", "Lung expansion": "numerical", "Smoking history": "boolean", "Other Malignancies": "boolean", "Previous surgeries": "boolean", "Infections": "boolean", "Trauma": "boolean", "Rheumatoid arthritis": "boolean", "Asbestos exposure": "boolean", "Occupational exposure": "boolean", "Environmental exposure": "boolean", "Chronic respiratory conditions": "boolean", "Asthma": "boolean", "Pleural effusion": "boolean", "Pneumoperitoneum": "boolean", "Atelectasis": "boolean", "Symptomatic pneumothorax": "boolean", "Tension pneumothorax": "boolean", "Weight loss": "boolean", "Decreased lung expansion": "boolean", "Pleural friction rub": "boolean", "Pleural fluid analysis": "boolean", "Blood tests": "boolean", "Chest radiography": "boolean", "Computed tomography (CT) scans": "boolean", "Pulmonary function tests": "boolean", "Heart failure": "boolean", "Perforation of the intestine": "boolean", "Surgical procedure": "boolean", "Pleural effusion risk factors (1-5)": "numerical", "Pleural disease risk factors (1-10)": "numerical"}\n\n\n## Healthy Patient Feature Range\n{"Elevated pericardial fluid levels (mL)": "0-200", "Increased pericardial fluid protein levels (mg/dL)": "0-5", "Measurements of cardiac diameter and thoracic diameter (cm)": "15-25", "Measurements of cardiac dimensions (cm)": "10-20", "White Blood Cell Count (cells/\\u03bcL)": "4,000-11,000", "C-Reactive Protein (CRP) levels (mg/L)": "0-10", "Procalcitonin levels (ng/mL)": "0-0.5", "Lung Volume (L)": "3-6", "Diffusing Capacity (mL/min/mmHg)": "20-40", "Gas Exchange (mL/min/mmHg)": "10-20", "Age (years)": "18-65", "Oxygen levels (mmHg)": "75-100", "Body Temperature (\\u00b0C)": "36-37", "Breathing Rate (breaths/min)": "12-20", "Cardiothoracic ratio": "0.45-0.55", "Lung expansion": "0-10", "Pleural effusion risk factors (1-5)": "1-5", "Pleural disease risk factors (1-10)": "1-10"}\n\n## Expected Output\n'

In [32]:
print(prompt)

You are a clinical expert. With following extra knowledge in mind:

**Summary of Lesions: Enlarged Cardiac Silhouette, Pulmonary Consolidation, Atelectasis, and Pleural Abnormality**

**Enlarged Cardiac Silhouette:**

*   Symptoms: shortness of breath, fatigue, swelling in the legs and feet, chest pain, palpitations, and coughing up blood
*   Causes: pericardial effusion, cardiomegaly, increased cardiothoracic ratio, anterior mediastinal mass, and prominent epicardial fat pad
*   Clinical signs: pericardial effusion, cardiomegaly, increased cardiothoracic ratio, anterior mediastinal mass, and prominent epicardial fat pad
*   Laboratory data: pericardial fluid analysis, radiographic images, echocardiographic images, and biopsy results
*   Personal relevant history: history of pericardial effusion and cardiomegaly, smoking history, and family history of heart disease

**Pulmonary Consolidation:**

*   Symptoms: breath sounds and crackles, chest pain, fever, weakness, shortness of breath,